In this case, we are given data from an audio book app. The condition to be included: Each customer in the database has made a purchase at least once. We want to create a machine learning algorithm to predict the likelihood of a customer making another purchase in the future.

In [47]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn import preprocessing 
# we will use the sklearn capablities to standardize the inputs
raw_csv_data=np.loadtxt("C:/Users/Yasmine Mohsen/Downloads/DataScienceCourse-datasets/Audiobooks_data.csv",delimiter=',')
unscaled_inputs_all=raw_csv_data[:,1:-1]
targets_all=raw_csv_data[:,-1]


### Preprocessing

#### Balance the dataset 
##### First, we must balance the dataset which is done by counting the total number of target 1s and matching the same number of target 0s to them. 

In [48]:
num_one_targets=int(np.sum(targets_all))
zero_targets_counter=0
indices_to_remove=[]

for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter+=1
        if zero_targets_counter>num_one_targets:
            indices_to_remove.append(i)
            # the above line is to remove the zero targets that are more than the number of ones
            
unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=  0)
targets_equal_priors=np.delete(targets_all,indices_to_remove,axis=0)          

#### Standardize the inputs

In [49]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)
# this will standardize the inputs to have zero mean and unit variance.

##### Shuffle the data

In [50]:
# since we will be using batch gradient descent, it's important to shuffle the data.

shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_equal_priors[shuffled_indices]

#### Split the dataset into training, validating and testing

In [51]:
samples_count=shuffled_inputs.shape[0]
train_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_samples_count=samples_count-train_samples_count-validation_samples_count

train_inputs=shuffled_inputs[:train_samples_count]
train_targets=shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count] 
validation_targets=shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets=shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)
# training on 3579, validation on 447, and testing on 448, total= 4474 (we started with 14084)
# all three sets are balanced (i.e., 50% of the target variable is 1)

1777.0 3579 0.4965074043028779
225.0 447 0.5033557046979866
235.0 448 0.5245535714285714


#### Save the three datasets for future use

In [52]:
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)

#### Create the machine learning algorithm

##### Data

In [53]:
npz=np.load('Audiobooks_data_train.npz')

train_inputs=npz['inputs'].astype(np.float32)
train_targets=npz['targets'].astype(np.int64)

npz=np.load('Audiobooks_data_validation.npz')
validation_inputs,validation_targets=npz['inputs'].astype(np.float32),npz['targets'].astype(np.int64)

npz=np.load('Audiobooks_data_test.npz')
test_inputs,test_targets=npz['inputs'].astype(np.float32),npz['targets'].astype(np.int64)


#### Module
Outline, optimizers, loss, early stopping and training

In [54]:
input_size=10
output_size=2
hidden_layer_size=50

model=tf.keras.Sequential([
  #  tf.keras.layers.flatten(input_shape=(28,28,1)),
  # ^ we can delete this line since we have already preprocessed our data properly
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax') 
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size=100
max_epochs=100

early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size, 
          epochs=max_epochs, 
          callbacks=[early_stopping],  # to stop training when the validation loss stops improving
          validation_data=(validation_inputs, validation_targets), 
          verbose=2)
# we notice that the validation accuracy is not strictly increasing. This indicates that the model is overfitting.
# so to prevent overfitting, we can use an early stopping callback.

Epoch 1/100


36/36 - 2s - 63ms/step - accuracy: 0.6362 - loss: 0.6374 - val_accuracy: 0.7136 - val_loss: 0.5314
Epoch 2/100
36/36 - 0s - 6ms/step - accuracy: 0.7538 - loss: 0.4903 - val_accuracy: 0.7830 - val_loss: 0.4413
Epoch 3/100
36/36 - 0s - 5ms/step - accuracy: 0.7826 - loss: 0.4327 - val_accuracy: 0.8121 - val_loss: 0.3957
Epoch 4/100
36/36 - 0s - 5ms/step - accuracy: 0.7888 - loss: 0.4013 - val_accuracy: 0.8098 - val_loss: 0.3745
Epoch 5/100
36/36 - 0s - 4ms/step - accuracy: 0.8019 - loss: 0.3829 - val_accuracy: 0.8210 - val_loss: 0.3620
Epoch 6/100
36/36 - 0s - 5ms/step - accuracy: 0.8036 - loss: 0.3711 - val_accuracy: 0.8233 - val_loss: 0.3515
Epoch 7/100
36/36 - 0s - 5ms/step - accuracy: 0.8106 - loss: 0.3605 - val_accuracy: 0.8143 - val_loss: 0.3480
Epoch 8/100
36/36 - 0s - 5ms/step - accuracy: 0.8080 - loss: 0.3550 - val_accuracy: 0.8166 - val_loss: 0.3416
Epoch 9/100
36/36 - 0s - 5ms/step - accuracy: 0.8150 - loss: 0.3491 - val_accuracy: 0.8255 - val_loss: 0.3408
Epoch 10/100
36/36 - 

#### Test the model

In [55]:
test_loss, test_accuracy=model.evaluate(test_inputs, test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7665 - loss: 0.3703  


In [56]:
print('Test loss: {0:.2f}, Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss: 0.33, Test accuracy: 80.13%
